In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 3, Finished, Available, Finished)

In [2]:
DeltaTable.createIfNotExists(spark)\
          .tableName("Gold_OrderPriority")\
          .addColumn("OrderPriority_ID",LongType())\
          .addColumn("Order_Priority",StringType())\
          .addColumn("Created_TS",TimestampType())\
          .addColumn("Modified_TS",TimestampType())\
          .execute()

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 4, Finished, Available, Finished)

In [3]:
df=spark.read.table("ECommerce_Lakehouse.gold_orderpriority")

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 5, Finished, Available, Finished)

In [4]:
Max_Date=df.selectExpr("coalesce(max(Modified_TS),'1900-01-01')").first()[0]

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 6, Finished, Available, Finished)

In [6]:
Max_Date

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 8, Finished, Available, Finished)

'1900-01-01'

In [7]:
df_bronze=spark.read.table("ECommerce_Lakehouse.bronze_sales")

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 9, Finished, Available, Finished)

In [8]:
df_bronze_mod=df_bronze.select("Order_Priority").where(col("Modified_TS")>Max_Date).drop_duplicates()

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 10, Finished, Available, Finished)

In [9]:
Max_ID=df.selectExpr("coalesce(max(OrderPriority_ID),0)").first()[0]

df_final=df_bronze_mod.withColumn("OrderPriority_ID",Max_ID+monotonically_increasing_id()+1)

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 11, Finished, Available, Finished)

In [10]:
df_gold_delta=DeltaTable.forPath(spark,"Tables/gold_orderpriority")
df_bonze_table=df_final

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 12, Finished, Available, Finished)

In [11]:
df_gold_delta.alias("gold")\
             .merge(\
               df_bonze_table.alias("bronze"),\
                "gold.Order_Priority==Bronze.Order_Priority"\
             )\
             .whenMatchedUpdate(\
                  set={
                    "gold.Modified_TS":current_timestamp()
                  }
             )\
             .whenNotMatchedInsert(\
                    values={
                         "gold.OrderPriority_ID":"Bronze.OrderPriority_ID",
                     "gold.Order_Priority":"Bronze.Order_Priority",
                     "gold.Created_TS":current_timestamp(),
                     "gold.Modified_TS":current_timestamp()
                    }
             )\
             .execute()


StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 13, Finished, Available, Finished)

In [12]:
%%sql

SELECT * FROM gold_orderpriority

StatementMeta(, d86e7ef9-4365-492a-acb5-77d445b3478c, 14, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>